## 1. Import libraries

In [1]:
#import libraries

import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML

from bs4 import BeautifulSoup # library to parse HTML and XML documents
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes #install folium for maps
import folium # plotting library

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

# # 2. Scrape data from Wikipedia page into a DataFrame

In [2]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_cities_in_Los_Angeles_County,_California")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table)) #Create a pandas dataframe.


la_df = pd.DataFrame(df[0])

In [3]:
la_df.head(10)

,City,Date incorporated,Population as of(2010 Census)
0,Agoura Hills,"December 8, 1982",20330
1,Alhambra,"July 11, 1903",83653
2,Arcadia,"August 5, 1903",56364
3,Artesia,"May 29, 1959",16522
4,Avalon,"June 26, 1913",3728
5,Azusa,"December 29, 1898",46361
6,Baldwin Park,"January 25, 1956",75390
7,Bell,"November 7, 1927",35477
8,Bell Gardens,"August 1, 1961",42072
9,Bellflower,"September 3, 1957",76616


In [4]:
la_df.shape

(88, 3)

In [11]:
la_df.drop(['Date incorporated'], axis = 1, inplace = True)

In [12]:
la_df.head(10)

,City,Population as of(2010 Census)
0,Agoura Hills,20330
1,Alhambra,83653
2,Arcadia,56364
3,Artesia,16522
4,Avalon,3728
5,Azusa,46361
6,Baldwin Park,75390
7,Bell,35477
8,Bell Gardens,42072
9,Bellflower,76616


In [13]:
la_df.shape

(88, 2)

## 3. Get the geographical coordinates

In [17]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

In [18]:
# define a function to get coordinates
def get_latlng(city):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Los Angeles, CA'.format(city))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [19]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(city) for city in la_df["City"].tolist() ]

In [20]:
coords

[[34.14611000000008, -118.77811999999994],
 [34.09370000000007, -118.12726999999995],
 [34.13614000000007, -118.03886999999997],
 [33.861140000000034, -118.07967999999994],
 [33.34411000000006, -118.32138999999995],
 [34.13361000000003, -117.90588999999994],
 [34.08526000000006, -117.96032999999994],
 [33.97977000000003, -118.18884999999995],
 [33.96998000000008, -118.14901999999995],
 [33.883260000000064, -118.12228999999996],
 [34.07346000000007, -118.40031999999997],
 [34.14531000000005, -117.97093999999998],
 [34.18182000000007, -118.30775999999997],
 [34.15778000000006, -118.63841999999994],
 [33.83161000000007, -118.26208999999994],
 [33.868660000000034, -118.06369999999998],
 [34.09641000000005, -117.71639999999996],
 [34.001190000000065, -118.15493999999995],
 [33.89569000000006, -118.22435999999999],
 [34.08743000000004, -117.88990999999999],
 [33.95983000000007, -118.17602],
 [34.02161000000007, -118.39620999999994],
 [34.000860000000046, -117.82906999999994],
 [33.9407700000

In [21]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [22]:
# merge the coordinates into the original dataframe
la_df['Latitude'] = df_coords['Latitude']
la_df['Longitude'] = df_coords['Longitude']

In [23]:
# check the neighborhoods and the coordinates
print(la_df.shape)
la_df

(88, 4)


,City,Population as of(2010 Census),Latitude,Longitude
0,Agoura Hills,20330,34.14611,-118.77812
1,Alhambra,83653,34.09370,-118.12727
2,Arcadia,56364,34.13614,-118.03887
3,Artesia,16522,33.86114,-118.07968
4,Avalon,3728,33.34411,-118.32139
5,Azusa,46361,34.13361,-117.90589
6,Baldwin Park,75390,34.08526,-117.96033
7,Bell,35477,33.97977,-118.18885
8,Bell Gardens,42072,33.96998,-118.14902
9,Bellflower,76616,33.88326,-118.12229


In [24]:
# save the DataFrame as CSV file
la_df.to_csv("la_df.csv", index=False)

## 4. Create a map of Los Angeles County with cities superimposed on top

In [25]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Los Angeles are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Los Angeles are 34.0536909, -118.2427666.


In [26]:
# create map of Los Angeles using latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(la_df['Latitude'], la_df['Longitude'], la_df['City']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_la)  
    
map_la

In [27]:
# save the map as HTML file
map_la.save('map_la.html')

## 5. Use the Foursquare API to explore the cities

In [28]:
CLIENT_ID = '302QNTO202TSXIILVEID2Y2R3A2WAQ5RHX22CWRSDSWYNUXE' # Foursquare ID
CLIENT_SECRET = 'BM1DD5YLS0KXDTXJUS44ZTUZEZBXRQYEGLHM004GPHD44VCT' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
NO_OF_VENUES = 100
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 302QNTO202TSXIILVEID2Y2R3A2WAQ5RHX22CWRSDSWYNUXE
CLIENT_SECRET:BM1DD5YLS0KXDTXJUS44ZTUZEZBXRQYEGLHM004GPHD44VCT


In [29]:
#get the top 100 venues that are within a radius of 2000 meters

venues = []

for lat, long, city in zip(la_df['Latitude'], la_df['Longitude'], la_df['City']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius,
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            city,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [30]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['City', 'Latitude', 'Longitude', 'Venue Name', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

print(venues_df.shape)
venues_df.head(10)

(7444, 7)


,City,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Agoura Hills,34.14611,-118.77812,Future Track Running Center,34.145819,-118.779251,Sporting Goods Shop
1,Agoura Hills,34.14611,-118.77812,Twisted Oak Tavern,34.145308,-118.778679,Gastropub
2,Agoura Hills,34.14611,-118.77812,Cafe Bizou,34.148410,-118.782587,French Restaurant
3,Agoura Hills,34.14611,-118.77812,Grissini Ristorante,34.145815,-118.778534,Italian Restaurant
4,Agoura Hills,34.14611,-118.77812,Pizza Nosh,34.148311,-118.782181,Pizza Place
5,Agoura Hills,34.14611,-118.77812,Forest Cove Park,34.152290,-118.774749,Park
6,Agoura Hills,34.14611,-118.77812,Target,34.148045,-118.793964,Big Box Store
7,Agoura Hills,34.14611,-118.77812,Hook Burger,34.147716,-118.796260,Burger Joint
8,Agoura Hills,34.14611,-118.77812,El Pollo Loco,34.144732,-118.761088,Fast Food Restaurant
9,Agoura Hills,34.14611,-118.77812,Ladyface Alehouse & Brasserie,34.143834,-118.762823,Brewery


In [31]:
venues_df.groupby(["City"]).count()

,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Agoura Hills,100,100,100,100,100,100
Alhambra,100,100,100,100,100,100
Arcadia,100,100,100,100,100,100
Artesia,100,100,100,100,100,100
Avalon,73,73,73,73,73,73
Azusa,92,92,92,92,92,92
Baldwin Park,90,90,90,90,90,90
Bell,88,88,88,88,88,88
Bell Gardens,74,74,74,74,74,74


In [32]:
print('There are {} uniques categories.'.format(len(venues_df['Venue Category'].unique())))

There are 345 uniques categories.


In [33]:
# print out the list of categories
venues_df['Venue Category'].unique()[:50]

array(['Sporting Goods Shop', 'Gastropub', 'French Restaurant',
       'Italian Restaurant', 'Pizza Place', 'Park', 'Big Box Store',
       'Burger Joint', 'Fast Food Restaurant', 'Brewery',
       'Ice Cream Shop', 'Liquor Store', 'New American Restaurant',
       'Indian Restaurant', 'Deli / Bodega', 'American Restaurant',
       'Food Court', 'Warehouse Store', 'Grocery Store',
       'Mexican Restaurant', 'Restaurant', 'Sushi Restaurant',
       'Mediterranean Restaurant', 'Breakfast Spot', 'Gym', 'Café',
       'Steakhouse', 'Asian Restaurant', 'Arts & Crafts Store',
       'Paper / Office Supplies Store', 'Music Store', 'Sandwich Place',
       'Donut Shop', 'Bakery', 'Dumpling Restaurant',
       'Seafood Restaurant', 'Department Store', 'Multiplex',
       'Thai Restaurant', 'Chinese Restaurant', 'Juice Bar', 'Lounge',
       'Laundry Service', 'Clothing Store', 'Coffee Shop', 'ATM',
       'Pharmacy', 'Golf Course', 'Sports Bar', 'Convenience Store'],
      dtype=object)

## 6. Analyze Each Neighborhood

In [34]:
# one hot encoding
la_onehot = pd.get_dummies(venues_df[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
la_onehot['City'] = venues_df['City'] 

# move city column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

print(la_onehot.shape)
la_onehot.head()

(7444, 346)


,City,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Castle,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Credit Union,Creperie,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Spot,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Outlet Mall,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shop

#### Group rows by city and by taking the mean of the frequency of occurrence of each category

In [35]:
la_grouped = la_onehot.groupby(["City"]).mean().reset_index()

print(la_grouped.shape)
la_grouped

(88, 346)


,City,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Castle,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Credit Union,Creperie,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Spot,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Outlet Mall,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shop

In [36]:
len(la_grouped[la_grouped["Café"] > 0])

48

#### Create a new DataFrame for Café data only

In [37]:
la_cafe = la_grouped[["City","Café"]]

In [38]:
la_cafe.head(10)

,City,Café
0,Agoura Hills,0.010000
1,Alhambra,0.040000
2,Arcadia,0.000000
3,Artesia,0.040000
4,Avalon,0.013699
5,Azusa,0.010870
6,Baldwin Park,0.000000
7,Bell,0.000000
8,Bell Gardens,0.000000
9,Bellflower,0.000000


## 7. Cluster Neighborhoods

In [39]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 4

la_clustering = la_cafe.drop(["City"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 1, 2, 0, 0, 1, 1, 1, 1], dtype=int32)

In [40]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each city.
la_merged = la_cafe.copy()

# add clustering labels
la_merged["Cluster Labels"] = kmeans.labels_

In [41]:
# merge la_grouped with la_data to add latitude/longitude for each city
la_merged = la_merged.join(la_df.set_index("City"), on="City")

print(la_merged.shape)
la_merged.head() # check the last columns!

(88, 6)


,City,Café,Cluster Labels,Population as of(2010 Census),Latitude,Longitude
0,Agoura Hills,0.010000,0,20330,34.14611,-118.77812
1,Alhambra,0.040000,2,83653,34.09370,-118.12727
2,Arcadia,0.000000,1,56364,34.13614,-118.03887
3,Artesia,0.040000,2,16522,33.86114,-118.07968
4,Avalon,0.013699,0,3728,33.34411,-118.32139


In [42]:
# sort the results by Cluster Labels
print(la_merged.shape)
la_merged.sort_values(["Cluster Labels"], inplace=True)
la_merged

(88, 6)


,City,Café,Cluster Labels,Population as of(2010 Census),Latitude,Longitude
0,Agoura Hills,0.010000,0,20330,34.14611,-118.77812
29,Glendora,0.013699,0,50073,34.13602,-117.86452
32,Hermosa Beach,0.010000,0,19506,33.86404,-118.39535
33,Hidden Hills,0.015873,0,1856,34.15918,-118.64025
34,Huntington Park,0.010000,0,58114,33.98143,-118.21914
36,Inglewood,0.010000,0,109673,33.96178,-118.35674
38,La Cañada Flintridge,0.010000,0,20246,34.20766,-118.20725
86,Westlake Village,0.012346,0,8270,34.14987,-118.81875
44,Lancaster,0.014493,0,160316,34.69890,-118.14478
28,Glendale,0.020000,0,203054,34.14633,-118.24864


#### Finally, let's visualize the resulting clusters

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged['Latitude'], la_merged['Longitude'], la_merged['City'], la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## 8. Examine Clusters

#### Cluster 0

In [45]:
la_merged.loc[la_merged['Cluster Labels'] == 0]

,City,Café,Cluster Labels,Population as of(2010 Census),Latitude,Longitude
0,Agoura Hills,0.010000,0,20330,34.14611,-118.77812
29,Glendora,0.013699,0,50073,34.13602,-117.86452
32,Hermosa Beach,0.010000,0,19506,33.86404,-118.39535
33,Hidden Hills,0.015873,0,1856,34.15918,-118.64025
34,Huntington Park,0.010000,0,58114,33.98143,-118.21914
36,Inglewood,0.010000,0,109673,33.96178,-118.35674
38,La Cañada Flintridge,0.010000,0,20246,34.20766,-118.20725
86,Westlake Village,0.012346,0,8270,34.14987,-118.81875
44,Lancaster,0.014493,0,160316,34.69890,-118.14478
28,Glendale,0.020000,0,203054,34.14633,-118.24864


#### Cluster 1

In [46]:
la_merged.loc[la_merged['Cluster Labels'] == 1]

,City,Café,Cluster Labels,Population as of(2010 Census),Latitude,Longitude
22,Diamond Bar,0.0,1,55544,34.00086,-117.82907
8,Bell Gardens,0.0,1,42072,33.96998,-118.14902
7,Bell,0.0,1,35477,33.97977,-118.18885
52,Maywood,0.0,1,27395,33.98757,-118.18981
56,Norwalk,0.0,1,105549,33.91681,-118.07252
6,Baldwin Park,0.0,1,75390,34.08526,-117.96033
61,Pico Rivera,0.0,1,62942,33.98242,-118.08799
62,Pomona,0.0,1,149058,34.05483,-117.75003
65,Rolling Hills,0.0,1,1860,33.77609,-118.34399
69,San Fernando,0.0,1,23645,34.28406,-118.43844


#### Cluster 2

In [47]:
la_merged.loc[la_merged['Cluster Labels'] == 2]

,City,Café,Cluster Labels,Population as of(2010 Census),Latitude,Longitude
85,West Hollywood,0.040000,2,34399,34.09069,-118.37053
82,Vernon,0.048780,2,112,34.00513,-118.23025
1,Alhambra,0.040000,2,83653,34.09370,-118.12727
74,Santa Monica,0.060000,2,89736,34.01158,-118.49227
3,Artesia,0.040000,2,16522,33.86114,-118.07968
70,San Gabriel,0.040000,2,39718,34.09708,-118.10824
71,San Marino,0.045455,2,13147,34.12208,-118.10521
68,San Dimas,0.050847,2,33371,34.10668,-117.80298


In [48]:
la_merged.loc[la_merged['Cluster Labels'] == 3]

,City,Café,Cluster Labels,Population as of(2010 Census),Latitude,Longitude
55,Monterey Park,0.026316,3,60269,34.05979,-118.12651
57,Palmdale,0.024390,3,152750,34.57923,-118.11657
21,Culver City,0.030000,3,38883,34.02161,-118.39621
53,Monrovia,0.030000,3,36590,34.14783,-117.99878
12,Burbank,0.030000,3,103340,34.18182,-118.30776
79,South Pasadena,0.030000,3,25619,34.11580,-118.15213
11,Bradbury,0.030769,3,1048,34.14531,-117.97094
58,Palos Verdes Estates,0.027778,3,13438,33.80003,-118.39187
66,Rolling Hills Estates,0.028571,3,8067,33.78355,-118.35383
24,Duarte,0.031746,3,21321,34.13961,-117.97050


### Results and Discussion:

After clustering data from the Foursquare API, we can see that cluster 1 is compromised of cities without cafés. This represents a great opportunity and high potential areas to open new cafés as there is no competition present. Meanwhile, cafés in cluster 2 are likely suffering from intense competition due to high concentration. Therefore, this project recommends property developers to open new cafés in cities in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new cafés in cities in cluster 0 with minimal competition. Lastly, property developers are advised to avoid cities in cluster 2 and cluster 3, which already have a moderate to high number of cafés.